In [1]:
# main_backtest.ipynb

# --- 0. 初期設定 & モジュールインポート ---
import backtrader as bt
import pandas as pd
import os
from datetime import datetime

# 作成したモジュールから必要なものをインポート
try:
    # .pyファイルと同じ階層にある場合
    from montecarlo_logic import DecompositionMonteCarloLogic
    from strategies import MonteCarloSmaCrossWithTPSL
    # from strategies import SmaCrossStrategy # 必要なら他の戦略も
except ImportError as e:
    print(f"モジュールのインポートに失敗しました: {e}")
    print("ipynbファイルと .py ファイルが同じディレクトリにあるか確認してください。")
    # 必要に応じて sys.path にパスを追加
    # import sys
    # sys.path.append('/path/to/your/modules')

print("モジュールをインポートしました。")

# --- 1. データ準備 ---
csv_file_path = 'USDJPYs_M10_data.csv'
if not os.path.exists(csv_file_path):
     raise FileNotFoundError(f"データファイルが見つかりません: {csv_file_path}")

dataframe = pd.read_csv(csv_file_path, index_col='datetime', parse_dates=True)
# データ期間を絞る場合 (オプション)
# dataframe = dataframe['2024-01-01':'2024-12-31']
data_feed = bt.feeds.PandasData(dataname=dataframe, name="USDJPYs_M10")
print(f"データ {csv_file_path} を読み込みました (期間: {dataframe.index.min()} - {dataframe.index.max()})。")

# --- 2. パラメータ設定 ---
initial_cash = 1000000.0
strategy_params = {
    'unit_lot': 0.01,
    'sma_short_period': 12,
    'sma_long_period': 26,
    'tp_pips': 25.0, # 対称性を試す例
    'sl_pips': 25.0, # 対称性を試す例
    # 'debug_log': True, # 詳細ログが必要な場合
}
print("バックテストパラメータを設定しました:", strategy_params)

# --- 3. バックテスト実行 ---
cerebro = bt.Cerebro()
cerebro.broker.setcash(initial_cash)
cerebro.adddata(data_feed)

# インポートしたStrategyクラスとパラメータ辞書を渡す
cerebro.addstrategy(MonteCarloSmaCrossWithTPSL, **strategy_params)

# アナライザー追加
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Days, riskfreerate=0.0)
print("アナライザーを追加しました。")

print("\nバックテストを実行します...")
results = cerebro.run()
strat = results[0]
print("バックテスト完了。")

# --- 4. 結果表示 ---
print("\n--- バックテスト結果 ---")
final_value = cerebro.broker.getvalue()
print(f"最終資金: {final_value:,.2f}")

# アナライザー結果の表示
try:
    ta = strat.analyzers.tradeanalyzer.get_analysis()
    dd = strat.analyzers.drawdown.get_analysis()
    sr = strat.analyzers.sharpe.get_analysis()

    if ta and hasattr(ta, 'total') and ta.total.closed > 0:
        total_closed_trades = ta.total.closed; total_pnl = ta.pnl.net.total
        win_rate = (ta.won.total / total_closed_trades * 100) if ta.won.total else 0
        avg_win = ta.won.pnl.average if ta.won.total > 0 else 0
        avg_loss = ta.lost.pnl.average if ta.lost.total > 0 else 0
        payoff_ratio = abs(avg_win / avg_loss) if avg_loss != 0 else float('inf')
        print(f"完了した総取引回数: {total_closed_trades}")
        print(f"総損益 (Net PnL): {total_pnl:.2f}")
        print(f"勝率: {win_rate:.2f}%")
        print(f"平均利益: {avg_win:.2f}")
        print(f"平均損失: {avg_loss:.2f}")
        print(f"ペイオフレシオ: {payoff_ratio:.2f}")
    else: print("完了した取引がありませんでした。")

    print(f"最大ドローダウン: {dd.max.drawdown:.2f}%")
    print(f"シャープレシオ (Annualized): {sr['sharperatio']:.3f}" if sr and 'sharperatio' in sr else "N/A")

except Exception as e: print(f"分析結果の表示中にエラー: {e}")

# --- 5. (オプション) グラフ表示 ---
# %matplotlib inline # Notebook内で表示する場合
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8] # サイズ調整
# cerebro.plot(style='candlestick') # プロット実行

モジュールをインポートしました。
データ USDJPYs_M10_data.csv を読み込みました (期間: 2023-01-02 01:00:00 - 2025-04-18 09:50:00)。
バックテストパラメータを設定しました: {'unit_lot': 0.01, 'sma_short_period': 12, 'sma_long_period': 26, 'tp_pips': 25.0, 'sl_pips': 25.0}
アナライザーを追加しました。

バックテストを実行します...
2025-04-18, Strategy Parameters: Unit Lot=0.01, SMA=12/26, TP=25.0pips, SL=25.0pips, PipValue=0.01
2025-04-18, Initial Monte Carlo Sequence: [0, 1]
2025-04-18, MonteCarloSmaCrossWithTPSL Initialized
2023-01-02, BUY BRACKET CREATE: Units=1, Size=0.01, SL=130.677, TP=131.177, Sequence: [0, 1]
2023-01-02, BUY Market Entry EXECUTED: Ref=1, Price: 130.927, Size: 0.01
2023-01-02, SELL EXECUTED: Ref=2, Price: 130.677, Size: -0.01
2023-01-02, Order Canceled: Ref=3
2023-01-02, Trade Closed (SL hit?): PNL=-0.00, Win=False, Traded Units=1, Seq Before: [0, 1]
2023-01-02,                                          Seq After: [0, 1, 1]
2023-01-02, BUY BRACKET CREATE: Units=1, Size=0.01, SL=130.523, TP=131.023, Sequence: [0, 1, 1]
2023-01-02, BUY Market E